# Classificação e Regressão com XGBoost

### Importando as bibliotecas que vão ser usadas 

instalações necessarias, obs: se já estiver instalados não é necessario fazer a instalação novamente 

In [ ]:
#!pip install xgboost

In [ ]:
#!pip install notebook pandas seaborn scikit-Learn 

In [12]:
# Importando as bibliotecas

import psycopg2 # chamando biblioteca 
import datetime

# Bibliotecas estatísticas
import statistics
import math
from scipy import stats
from collections import Counter

# Bibliotecas 
import seaborn as sns
import plotly.express as px  #gráficos interativos 


# Usado para o processamento de arrays e matrizes
import numpy as np 

# Usado para criar e manipular dataframes
import pandas as pd 

# Usado para criar gráficos
import matplotlib.pyplot as plt #plotando gráficos 2d

# Usado para tornar a ordem dos dados aleatória
from sklearn.utils import shuffle 



#bibliotecas 
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import xgboost as xgb


## Carregando os dados para regressão e classificação 
O foco vai ser utilizar o XGBoost mais que nos problemas  de regressão e classificação em si. 

In [7]:
import seaborn as sns

df_iris = sns.load_dataset('iris') #carregando dados
df_iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [8]:
df_iris.shape # número de observações e números de variáveis 

df_iris['species'].unique() #mostra as especeis de orquidias 

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [9]:
#codificação das especeis de orquidias 
df_iris['species'] = df_iris['species'].map({'setosa': 0, 'versicolor': 1, 'virginica': 2})
df_iris['species'].unique() #visualiza a codificação 

array([0, 1, 2], dtype=int64)

In [13]:
#Separando as bases 

x_iris = df_iris.drop(['species'],axis=1)
y_iris = df_iris['species']

#sepando em treino e teste 
x_iris_treino, x_iris_teste, y_iris_treino,y_iris_teste = train_test_split(x_iris, y_iris ,random_state=1)

In [20]:
# Criando o objeto com o classificador XGBoost
classificador_xgb = xgb.XGBClassifier() #não foi mudando nenhum hiperparametro 

type(classificador_xgb) #verificando o tipo de objeto que foi criado 

xgboost.sklearn.XGBClassifier

### Compatibilidade com scikit-Learn

-> o XGBoosté uma subclasse de sklearn.Base.classifierMixin, ou seja, é compativel com sklearn. É possível então utilizar o arcabouço do scikit-learn, como métodos de CV, pipelines, etc....

In [21]:
from sklearn.model_selection import cross_val_score # calcula um Cross Validation, validação cruzada

# cross_val_score serve para comparar modelos, dar uma ideia de como o modelo está performando 


# criando 
resultado_cv = cross_val_score(classificador_xgb, x_iris_treino, y_iris_treino)
print("resultado_cv:",resultado_cv)

#calculndo Acurácia 
acuracia_cv = resultado_cv.mean()
print("Acurácia:",acuracia_cv*100) #em porcentagem 


resultado_cv: [0.95652174 0.95652174 0.95454545 1.         0.86363636]
Acurácia: 94.62450592885375


<a> Tunagem </a>

o  XGBoost  permite alteração de vários hiperparâmetros, como learning rate, altura máxima das árvores, quantidade de folhas nas árvores, número de árvores (estimadores), etc...

 Nosso foco não é em tunagem de hiperparâmetros, mas vamos muudar somente a altura máxima.

 Vamos aproveitar também  para mostrar uma característica interessante de XGBoost. Ele permite alterar a implementação do classificador. Podemos, por exemplo. Utilizar florest (bagging) ao inves de algoritmos de boosting! vamos ver se melhora o resultado 

 -> outro hiperparâmetro importante é a função de erro utilizada para otimização: objective

 como o padrão para classificação é "binary:logistc" e para regressão "reg:squarederror". Mas quando passamos uma variável target com mais de duas classes ele muda automaticamente para "multi:softprob". vamos ver na prática!

In [24]:
clf = xgb.XGBClassifier() #sem nada 
print("antes: ", clf.objective)


clf = xgb.XGBClassifier().fit(x_iris_treino,y_iris_treino)
print("depois muda automatico:",clf.objective)

antes:  binary:logistic
depois muda automatico: multi:softprob


Poderiamos também utilizar a multi:softmaz, mas esta esta somente retorna a classe com maior probabilidade, então  multi:softprob entrega as probabilidades de todas as classes para cada predição 

In [25]:
# max_depth altua maxima das arvores de boosting, elas podem ter no máximo altura de 2 da raiz até as folhas 

classificador_xgb_tunado = xgb.XGBClassifier(max_depth = 2) 

100* cross_val_score(classificador_xgb_tunado,x_iris_treino,y_iris_treino).mean()

95.53359683794467

## Tipos de classificador 

'gbtree','gblinear', ou 'dart'(gbtree e dart usam modelos baseados em árvore enquanto gblinear usa funções lineares)

In [31]:
#usando o classificador dart
classificador_xgb_dart = xgb.XGBClassifier(booster = 'dart')
dart = 100* cross_val_score(classificador_xgb_dart,x_iris_treino,y_iris_treino).mean()
print(" Acurácia usando dart:", dart)

#usando o classificador gblinear
classificador_xgb_gblinear = xgb.XGBClassifier(booster = 'gblinear')
gblinear = 100* cross_val_score(classificador_xgb_gblinear,x_iris_treino,y_iris_treino).mean()
print("Acurácia usando gblinear:", gblinear)

 Acurácia usando dart: 94.62450592885375
Acurácia usando gblinear: 96.40316205533598


# Classificando com o modelo Campeão 🏆

In [33]:
#com o melhor modelo, podemos utilizar a base toda de treino 
classificador_campeao = classificador_xgb_gblinear

classificador_campeao.fit(x_iris_treino,y_iris_treino)

XGBClassifier(base_score=None, booster='gblinear', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

 obs: só depois de ter um modelo campeão, treinei na base de treino. agora posso fazer predições na base de teste 

In [34]:
# podemos realizar a predição da base de teste!

predicoes_iris = classificador_campeao.predict(x_iris_teste)
predicoes_iris

array([0, 1, 1, 0, 2, 1, 2, 0, 0, 2, 1, 0, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       2, 0, 2, 1, 0, 0, 1, 2, 1, 2, 1, 2, 2, 0, 1, 0], dtype=int64)

In [35]:
y_iris_teste #veriicado como ta ficando essa classificação 

14     0
98     1
75     1
16     0
131    2
56     1
141    2
44     0
29     0
120    2
94     1
5      0
102    2
51     1
78     1
42     0
92     1
66     1
31     0
35     0
90     1
84     1
77     1
40     0
125    2
99     1
33     0
19     0
73     1
146    2
91     1
135    2
69     1
128    2
114    2
48     0
53     1
28     0
Name: species, dtype: int64

In [41]:
# Calculando o número de acertos 
acertos = (predicoes_iris == y_iris_teste).sum()
print("acertos:",acertos)

# Mas qual o tamanho da base de teste ?
total_observacoes = len(y_iris_teste)
print("tamanho da base de teste:", total_observacoes) 


# Mas qual o tamanho da base de teste ?
print("acuracia:", 100* (acertos/total_observacoes))

#verificando a qualidade do modelo (calcula de forma correta)
from sklearn.metrics import classification_report #verifica a qualidade do  modelo 
print('\n verificando a qualidade do modelo: \n', classification_report(y_iris_teste, predicoes_iris)) 

# Gerando a matriz de confusão do modelo 
from sklearn.metrics import confusion_matrix #gera a matriz de confusão do modelo
print('Matriz de confusão: \n',confusion_matrix(y_iris_teste, predicoes_iris))

acertos: 37
tamanho da base de teste: 38
acuracia: 97.36842105263158

 verificando a qualidade do modelo: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       1.00      0.94      0.97        16
           2       0.90      1.00      0.95         9

    accuracy                           0.97        38
   macro avg       0.97      0.98      0.97        38
weighted avg       0.98      0.97      0.97        38

Matriz de confusão: 
 [[13  0  0]
 [ 0 15  1]
 [ 0  0  9]]


# REGRESSÃO 

similar ao codigo anterior, a unica mudança vai ser de XGBClassifier para XGBBegressor

In [44]:
# vamos tentar prever o consumo dos carros a partir de suas features 
df_mpg = sns.load_dataset('mpg')
df_mpg.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,usa,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,usa,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,usa,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,usa,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,usa,ford torino


In [45]:
df_mpg.shape

(398, 9)

In [46]:
df_mpg.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000


In [48]:
#Separando as bases 
# tirando oringin e name 
x_mpg = df_mpg.drop(['mpg','origin','name'], axis=1)
y_mpg = df_mpg['mpg']

#sepando em treino e teste 
x_mpg_treino, x_mpg_teste, y_mpg_treino,y_mpg_teste = train_test_split(x_mpg, y_mpg ,random_state=1)

Lembrando que existe o hiperparâmetro objective que determina a função de erro a ser utilizada para otimização para otimização. 


In [49]:
# -> somente chegando o padrão para a regressão 
clf = xgb.XGBRegressor()
clf.objective

'reg:squarederror'

In [59]:
# modelo padrão sem tunagem de hipeparâmetros 
regressor_xgb = xgb.XGBRegressor()
#media de erros: 
padrao = cross_val_score(regressor_xgb, x_mpg_treino, y_mpg_treino, scoring='neg_root_mean_squared_error').mean()
print("media de erros modelo padrão sem tunagem de hipeparâmetros:", padrao*-1)

# modelo com altura maxima de 2 das arvores
regressor_xgb = xgb.XGBRegressor( max_depth = 2)
#media de erros: 
max_arvore = cross_val_score(regressor_xgb, x_mpg_treino, y_mpg_treino, scoring='neg_root_mean_squared_error').mean()
print("media de erros modelo com altura maxima de 2 das arvores:", max_arvore*-1)

# modelo usando o regressor gblinear
regressor_xgb = xgb.XGBRegressor(booster = 'gblinear')
#media de erros: 
regressor = cross_val_score(regressor_xgb, x_mpg_treino, y_mpg_treino, scoring='neg_root_mean_squared_error').mean()
print("media de erros modelo usando o regressor gblinear:", regressor*-1)

# -> utilize o que tiver menor valor! 

media de erros modelo padrão sem tunagem de hipeparâmetros: 3.1295619740712164
media de erros modelo com altura maxima de 2 das arvores: 3.0339315587321383
media de erros modelo usando o regressor gblinear: 4.416539277140776


In [65]:
regressor_campeao = regressor_xgb
regressor_campeao = xgb.XGBRegressor(eval_metric = 'rmsle')
regressor_campeao.fit(x_mpg_treino, y_mpg_treino)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmsle', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [66]:
predicoes_mpg = regressor_campeao.predict(x_mpg_teste)
predicoes_mpg[:10]

array([20.64906 , 28.883665, 16.86001 , 21.34499 , 20.39676 , 15.01599 ,
       31.683403, 37.18937 , 15.918009, 13.198436], dtype=float32)

In [67]:
y_mpg_teste[:10]

174    18.0
359    28.1
250    19.4
274    20.3
283    20.2
232    16.0
117    29.0
348    37.7
189    15.5
106    12.0
Name: mpg, dtype: float64

In [78]:
# calculando o erro  entre o que os valores queo modelo previu e os valores reais
from sklearn.metrics import mean_squared_error
import math

mse = mean_squared_error(y_mpg_teste,predicoes_mpg)
print('erro ao quadrado:', mse)

rmse = math.sqrt(mse)
print('erro: ', rmse)


erro ao quadrado: 8.665794944735287
erro:  2.943772230444347
